In [ ]:
import json
import os
from http.server import HTTPServer, BaseHTTPRequestHandler
from typing import List, Dict, Any, Optional
import time


class Task:
    def __init__(self, title: str, priority: str, task_id: Optional[str] = None, is_done: bool = False):
        self.title = title
        self.priority = priority
        self.is_done = is_done
        self.id = task_id or self._generate_id()

    def _generate_id(self) -> int:
        return int(time.time() * 1000)

    def to_dict(self) -> Dict[str, Any]:
        return {
            "id": self.id,
            "title": self.title,
            "priority": self.priority,
            "isDone": self.is_done
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Task':
        # Создает объект задачи из словаря
        return cls(
            title=data["title"],
            priority=data["priority"],
            task_id=data.get("id"),
            is_done=data.get("isDone", False)
        )


class TaskManager:
    def __init__(self, filename: str = "tasks.txt"):
        self.filename = filename
        self.tasks: List[Task] = []
        self.load_tasks()

    def load_tasks(self) -> None:
        # Загружает задачи из файла при старте сервера
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    self.tasks = [Task.from_dict(task_data) for task_data in data]
            except (json.JSONDecodeError, IOError):
                # Файл поврежден - начинаем с чистого списка
                self.tasks = []
        else:
            self.tasks = []

    def save_tasks(self) -> None:
        # Сохраняет все задачи в файл после каждого изменения
        try:
            tasks_data = [task.to_dict() for task in self.tasks]
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(tasks_data, f, ensure_ascii=False, indent=2)
        except IOError as e:
            print(f"Ошибка при сохранении задач: {e}")

    def create_task(self, title: str, priority: str) -> Task:
        # Создает новую задачу и добавляет в список
        valid_priorities = ["low", "normal", "high"]
        if priority not in valid_priorities:
            priority = "normal"

        task = Task(title=title, priority=priority)
        self.tasks.append(task)
        self.save_tasks()
        return task

    def get_all_tasks(self) -> List[Task]:
        # Возвращает все задачи
        return self.tasks

    def get_task_by_id(self, task_id: int) -> Optional[Task]:
        # Ищет задачу по идентификатору
        for task in self.tasks:
            if task.id == task_id:
                return task
        return None

    def mark_task_complete(self, task_id: int) -> bool:
        # Помечает задачу как выполненную
        task = self.get_task_by_id(task_id)
        if task:
            task.is_done = True
            self.save_tasks()
            return True
        return False


class TodoRequestHandler(BaseHTTPRequestHandler):
    # Менеджер задач общий для всех запросов
    task_manager = TaskManager()

    def _set_headers(self, status_code: int = 200, content_type: str = "application/json") -> None:
        # Устанавливает HTTP-заголовки ответа
        self.send_response(status_code)
        self.send_header('Content-type', content_type)
        self.send_header('Access-Control-Allow-Origin', '*')
        self.end_headers()

    def _send_json_response(self, data: Any, status_code: int = 200) -> None:
        # Отправляет JSON-ответ клиенту
        self._set_headers(status_code)
        response = json.dumps(data, ensure_ascii=False)
        self.wfile.write(response.encode('utf-8'))

    def _parse_json_body(self) -> Dict[str, Any]:
        # Читает и парсит JSON из тела запроса
        content_length = int(self.headers.get('Content-Length', 0))
        if content_length > 0:
            body = self.rfile.read(content_length).decode('utf-8')
            try:
                return json.loads(body)
            except json.JSONDecodeError:
                return {}
        return {}

    def do_GET(self) -> None:
        # Обрабатывает GET-запросы
        if self.path == '/tasks':
            # Возвращает список всех задач
            tasks = self.task_manager.get_all_tasks()
            tasks_data = [task.to_dict() for task in tasks]
            self._send_json_response(tasks_data)
        else:
            # Путь не найден
            self._send_json_response({"error": "Not Found"}, 404)

    def do_POST(self) -> None:
        # Обрабатывает POST-запросы
        if self.path == '/tasks':
            # Создает новую задачу
            data = self._parse_json_body()

            # Проверяет наличие обязательного поля title
            if 'title' not in data:
                self._send_json_response({"error": "Title is required"}, 400)
                return

            title = data['title']
            priority = data.get('priority', 'normal')

            # Создает задачу и возвращает её данные
            task = self.task_manager.create_task(title, priority)
            self._send_json_response(task.to_dict(), 201)

        elif self.path.startswith('/tasks/') and self.path.endswith('/complete'):
            # Помечает задачу как выполненную
            try:
                # Извлекает ID задачи из URL
                path_parts = self.path.split('/')
                task_id = int(path_parts[2])

                # Пытается найти и обновить задачу
                if self.task_manager.mark_task_complete(task_id):
                    self._set_headers(200)
                    self.wfile.write(b'')
                else:
                    # Задача с таким ID не найдена
                    self._send_json_response({"error": "Task not found"}, 404)

            except (ValueError, IndexError):
                # Некорректный формат ID или URL
                self._send_json_response({"error": "Invalid task ID"}, 400)
        else:
            # Путь не найден
            self._send_json_response({"error": "Not Found"}, 404)

    def do_OPTIONS(self) -> None:
        # Обрабатывает CORS preflight-запросы
        self.send_response(200)
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        self.end_headers()

    def log_message(self, format: str, *args: Any) -> None:
        # Выводит информацию о запросе в консоль
        print(f"{self.address_string()} - - [{self.log_date_time_string()}] {format % args}")


def run_server(port: int = 8080) -> None:
    # Создает и запускает HTTP-сервер
    server_address = ('', port)
    httpd = HTTPServer(server_address, TodoRequestHandler)

    # Выводит информацию о запуске сервера
    print(f"Сервер запущен на порту {port}")
    print(f"Доступные эндпоинты:")
    print(f"  GET  http://localhost:{port}/tasks - получить все задачи")
    print(f"  POST http://localhost:{port}/tasks - создать задачу")
    print(f"  POST http://localhost:{port}/tasks/<id>/complete - отметить задачу как выполненную")
    print(f"Файл для хранения задач: tasks.txt")
    print("Нажмите Ctrl+C для остановки сервера")

    try:
        # Запускает сервер в бесконечном цикле обработки запросов
        httpd.serve_forever()
    except KeyboardInterrupt:
        # Корректно останавливает сервер при нажатии Ctrl+C
        print("\nОстановка сервера...")
        httpd.server_close()


if __name__ == '__main__':
    # Точка входа в приложение
    run_server(8080)